<a href="https://colab.research.google.com/github/danielelbrecht/CAP5610-HW-1/blob/master/CAP5610_HW1Problem3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Single layer digit classifier using softmax activation and categorical cross entropy loss

In [0]:
#Import data
import numpy as np
import sklearn
from keras.datasets import mnist

#Load data
(train_images_original, train_labels_original), (test_images_original, test_labels_original) = mnist.load_data()

In [0]:
#Preprocess data
train_images = train_images_original.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images_original.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255



#Use one-hot encoding
from keras.utils import to_categorical

train_labels = to_categorical(train_labels_original)
test_labels = to_categorical(test_labels_original)

In [0]:
#softmax function
def softmax():
  
  return

In [0]:
import random

#Initialize weight matrix
#w[0][50] represents the 50th weight of the 0th neuron
weights = []
bias = []

for i in range(10):
  weights.append([])
  bias.append(random.random()-0.5)
  for j in range(784):
    weights[i].append(random.random()-0.5)